<a href="https://colab.research.google.com/github/Sabri-Nang/analisis_datos/blob/version1/analisis_estadisticas_criminales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Datos extraídos de:
[datos.gob.ar](https://www.datos.gob.ar/dataset/seguridad-snic---departamental-estadisticas-criminales-republica-argentina-por-departamentos/archivo/seguridad_3.2)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ruta_csv = '/content/drive/MyDrive/proyectos_análisis_datos/snic-departamentos-anual.csv'

In [5]:
# importo las librerías
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [6]:
df = pd.read_csv(ruta_csv, delimiter=';')

In [7]:
# muestra
df.sample(5)

,provincia_id,provincia_nombre,departamento_id,departamento_nombre,anio,codigo_delito_snic_id,cod_delito,codigo_delito_snic_nombre,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
495698,94,"Tierra del Fuego, Antártida e Islas del Atlánt...",94014,Ushuaia,2016,32,32,Delitos contra el orden económico y financiero,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
20615,6,Buenos Aires,6049,Azul,2005,23,23,Delitos contra el orden público,1,NaN,NaN,NaN,NaN,1.539480,NaN,NaN,NaN
439602,82,Santa Fe,82056,Iriondo,2014,23,23,Delitos contra el orden público,1,NaN,NaN,NaN,NaN,1.434905,NaN,NaN,NaN
58096,6,Buenos Aires,6315,General Juan Madariaga,2014,25,25,Delitos contra los poderes públicos y el orden...,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
155529,10,Catamarca,10112,Valle Viejo,2017,22_1,221,Fabricación adquisición transferencia y tenenc...,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497037 entries, 0 to 497036
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   provincia_id               497037 non-null  int64  
 1   provincia_nombre           497037 non-null  object 
 2   departamento_id            497037 non-null  int64  
 3   departamento_nombre        497037 non-null  object 
 4   anio                       497037 non-null  int64  
 5   codigo_delito_snic_id      497037 non-null  object 
 6   cod_delito                 497037 non-null  int64  
 7   codigo_delito_snic_nombre  497037 non-null  object 
 8   cantidad_hechos            497037 non-null  int64  
 9   cantidad_victimas          176027 non-null  float64
 10  cantidad_victimas_masc     176027 non-null  float64
 11  cantidad_victimas_fem      176027 non-null  float64
 12  cantidad_victimas_sd       176027 non-null  float64
 13  tasa_hechos                47

In [9]:
df.shape

(497037, 17)

In [10]:
df.columns

Index(['provincia_id', 'provincia_nombre', 'departamento_id',
       'departamento_nombre', 'anio', 'codigo_delito_snic_id', 'cod_delito',
       'codigo_delito_snic_nombre', 'cantidad_hechos', 'cantidad_victimas',
       'cantidad_victimas_masc', 'cantidad_victimas_fem',
       'cantidad_victimas_sd', 'tasa_hechos', 'tasa_victimas',
       'tasa_victimas_masc', 'tasa_victimas_fem'],
      dtype='object')

In [11]:
# veamos si codigo_delito_snic_id	 es igual a cod_delito

df['codigo_delito_snic_id'].astype(pd.Int64Dtype()).isnull().sum()

0

In [12]:
(df['codigo_delito_snic_id'].astype(pd.Int64Dtype()) != df['cod_delito']).sum() # ambas columnas son iguales

0

In [13]:
len(df['cod_delito'].unique()) # cantidad de codigos distintos

72

In [14]:
len(df['codigo_delito_snic_nombre'].unique())

72

In [15]:
# quiero ver si cada cod_delito corresponde a un codigo_delito_snic_nombre, o hay mas de uno para el mismo codigo
df.groupby('cod_delito')['codigo_delito_snic_nombre'].nunique().reset_index()

,cod_delito,codigo_delito_snic_nombre
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
67,295,2
68,296,2
69,297,1
70,298,2


In [16]:
# veamos los casos que un codigo engloba mas de un delito
codigos_delitos = df.groupby('cod_delito')['codigo_delito_snic_nombre'].unique().reset_index()

In [17]:
codigos_delitos['cantidad_delitos'] = codigos_delitos['codigo_delito_snic_nombre'].apply(len)

In [18]:
codigos_delitos

,cod_delito,codigo_delito_snic_nombre,cantidad_delitos
0,1,[Homicidios dolosos],1
1,2,[Homicidios dolosos en grado de tentativa],1
2,3,[Muertes en accidentes viales],1
3,4,[Homicidios culposos por otros hechos],1
4,5,[Lesiones dolosas],1
...,...,...,...
67,295,"[Contrabando Simple, Contrabando simple]",2
68,296,"[Contrabando Agravado, Contrabando agravado]",2
69,297,[Contrabando de elementos nucleares agresivos ...,1
70,298,"[Otros delitos previstos en leyes especiales, ...",2


In [19]:
distintos = codigos_delitos[codigos_delitos['cantidad_delitos'] > 1]
distintos

,cod_delito,codigo_delito_snic_nombre,cantidad_delitos
16,17,[Robos agravados por el resultado de lesiones ...,2
67,295,"[Contrabando Simple, Contrabando simple]",2
68,296,"[Contrabando Agravado, Contrabando agravado]",2
70,298,"[Otros delitos previstos en leyes especiales, ...",2


In [20]:
# hay diferencias en los formatos, espacios, mayusculas
# formateo sacando espacios y cambiando todo a minuscula
df['codigo_delito_snic_nombre'] = df['codigo_delito_snic_nombre'].str.strip()

In [21]:
df['codigo_delito_snic_nombre'] = df['codigo_delito_snic_nombre'].str.lower()

In [22]:
# verifico
codigos_delitos = df.groupby('cod_delito')['codigo_delito_snic_nombre'].unique().reset_index()
codigos_delitos['cantidad_delitos'] = codigos_delitos['codigo_delito_snic_nombre'].apply(len)
codigos_delitos[codigos_delitos['cantidad_delitos'] > 1]

,cod_delito,codigo_delito_snic_nombre,cantidad_delitos
70,298,"[otros delitos previstos en leyes especiales, ...",2


In [23]:
codigos_delitos[codigos_delitos['cantidad_delitos'] > 1]['codigo_delito_snic_nombre'].iloc[0]

array(['otros delitos previstos en leyes especiales',
       'otros delitos previstos en leyes especiales n.c.p'], dtype=object)

In [24]:
# reemplazo 'otros delitos previstos en leyes especiales n.c.p' por 'otros delitos previstos en leyes especiales'
df['codigo_delito_snic_nombre'] = df['codigo_delito_snic_nombre'].replace('otros delitos previstos en leyes especiales n.c.p', 'otros delitos previstos en leyes especiales')

In [25]:
# distintas provincias
print(len(df['provincia_nombre'].unique()))
print(len(df['provincia_id'].unique()))

24
24


In [26]:
df['provincia_nombre'].unique()

array(['Ciudad Autónoma de Buenos Aires', 'Buenos Aires', 'Catamarca',
       'Córdoba', 'Corrientes', 'Chaco', 'Chubut', 'Entre Ríos',
       'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 'Mendoza', 'Misiones',
       'Neuquén', 'Río Negro', 'Salta', 'San Juan', 'San Luis',
       'Santa Cruz', 'Santa Fe', 'Santiago del Estero', 'Tucumán',
       'Tierra del Fuego, Antártida e Islas del Atlántico Sur'],
      dtype=object)

In [27]:
# como variaron los delitos por año
# agrupo por año y delito
df.groupby(['anio', 'cod_delito', 'codigo_delito_snic_nombre']).aggregate({'cantidad_hechos': 'sum'})

cantidad_hechos
anio cod_delito codigo_delito_snic_nombre                                          
2000 1          homicidios dolosos                                             2653
     2          homicidios dolosos en grado de tentativa                        884
     3          muertes en accidentes viales                                   3686
     4          homicidios culposos por otros hechos                           1058
     5          lesiones dolosas                                             107864
...                                                                             ...
2023 295        contrabando simple                                            17898
     296        contrabando agravado                                            448
     297        contrabando de elementos nucleares agresivos qu...                7
     298        otros delitos previstos en leyes especiales                   40778
     2810       otros delitos previstos en la ley 23.737                       7432

[938 rows x 1 columns]

In [28]:
anios_cantidad = df.groupby(['anio', 'cod_delito', 'codigo_delito_snic_nombre']).aggregate({'cantidad_hechos': 'sum'}).reset_index()
anios_cantidad

,anio,cod_delito,codigo_delito_snic_nombre,cantidad_hechos
0,2000,1,homicidios dolosos,2653
1,2000,2,homicidios dolosos en grado de tentativa,884
2,2000,3,muertes en accidentes viales,3686
3,2000,4,homicidios culposos por otros hechos,1058
4,2000,5,lesiones dolosas,107864
...,...,...,...,...
933,2023,295,contrabando simple,17898
934,2023,296,contrabando agravado,448
935,2023,297,contrabando de elementos nucleares agresivos q...,7
936,2023,298,otros delitos previstos en leyes especiales,40778


In [29]:
maximos_delitos_anuales = anios_cantidad.groupby('anio')['cantidad_hechos'].max().reset_index()
maximos_delitos_anuales

,anio,cantidad_hechos
0,2000,320649
1,2001,345725
2,2002,423478
3,2003,393547
4,2004,352554
5,2005,322267
6,2006,318613
7,2007,331723
8,2008,361716
9,2009,367975


In [30]:
# merge para obtener delitos
pd.merge(anios_cantidad, maximos_delitos_anuales, on=['anio', 'cantidad_hechos'])

,anio,cod_delito,codigo_delito_snic_nombre,cantidad_hechos
0,2000,15,robos (excluye los agravados por el resultado ...,320649
1,2001,15,robos (excluye los agravados por el resultado ...,345725
2,2002,15,robos (excluye los agravados por el resultado ...,423478
3,2003,15,robos (excluye los agravados por el resultado ...,393547
4,2004,15,robos (excluye los agravados por el resultado ...,352554
5,2005,15,robos (excluye los agravados por el resultado ...,322267
6,2006,15,robos (excluye los agravados por el resultado ...,318613
7,2007,15,robos (excluye los agravados por el resultado ...,331723
8,2008,15,robos (excluye los agravados por el resultado ...,361716
9,2009,15,robos (excluye los agravados por el resultado ...,367975


In [31]:
df['codigo_delito_snic_nombre'].unique()

array(['homicidios dolosos',
       'abusos sexuales con acceso carnal (violaciones)',
       'otros delitos contra la integridad sexual',
       'delitos contra el estado civil', 'amenazas',
       'otros delitos contra la libertad',
       'robos (excluye los agravados por el resultado de lesiones y/o muertes)',
       'tentativas de robo (excluye las agravadas por el res. de lesiones y/o muerte)',
       'robos agravados por el resultado de lesiones y/o muertes',
       'tentativas de robo agravado por el resultado de lesiones y/o muertes',
       'hurtos', 'homicidios dolosos en grado de tentativa',
       'tentativas de hurto', 'otros delitos contra la propiedad',
       'delitos contra la seguridad pública',
       'delitos contra el orden público',
       'delitos contra la seguridad de la nación',
       'delitos contra los poderes públicos y el orden constitucional',
       'delitos contra la administración pública',
       'delitos contra la fe pública', 'ley 23.737 (estupefa

# Analizo los homicidios

In [32]:
df.columns

Index(['provincia_id', 'provincia_nombre', 'departamento_id',
       'departamento_nombre', 'anio', 'codigo_delito_snic_id', 'cod_delito',
       'codigo_delito_snic_nombre', 'cantidad_hechos', 'cantidad_victimas',
       'cantidad_victimas_masc', 'cantidad_victimas_fem',
       'cantidad_victimas_sd', 'tasa_hechos', 'tasa_victimas',
       'tasa_victimas_masc', 'tasa_victimas_fem'],
      dtype='object')

In [34]:
df['codigo_delito_snic_nombre'].unique()

array(['homicidios dolosos',
       'abusos sexuales con acceso carnal (violaciones)',
       'otros delitos contra la integridad sexual',
       'delitos contra el estado civil', 'amenazas',
       'otros delitos contra la libertad',
       'robos (excluye los agravados por el resultado de lesiones y/o muertes)',
       'tentativas de robo (excluye las agravadas por el res. de lesiones y/o muerte)',
       'robos agravados por el resultado de lesiones y/o muertes',
       'tentativas de robo agravado por el resultado de lesiones y/o muertes',
       'hurtos', 'homicidios dolosos en grado de tentativa',
       'tentativas de hurto', 'otros delitos contra la propiedad',
       'delitos contra la seguridad pública',
       'delitos contra el orden público',
       'delitos contra la seguridad de la nación',
       'delitos contra los poderes públicos y el orden constitucional',
       'delitos contra la administración pública',
       'delitos contra la fe pública', 'ley 23.737 (estupefa

In [43]:
# cuantos contienen la palabra homicidio
df[df['codigo_delito_snic_nombre'].str.contains('homicidio')]['codigo_delito_snic_nombre']

,codigo_delito_snic_nombre
0,homicidios dolosos
11,homicidios dolosos en grado de tentativa
25,homicidios culposos por otros hechos
31,homicidios dolosos
42,homicidios dolosos en grado de tentativa
...,...
496928,homicidios dolosos en grado de tentativa
496965,homicidios culposos por otros hechos
496971,homicidios dolosos
496988,homicidios dolosos en grado de tentativa


In [44]:
df_homicidios = df[df['codigo_delito_snic_nombre'].str.contains('homicidio')]
df_homicidios.sample(5)

,provincia_id,provincia_nombre,departamento_id,departamento_nombre,anio,codigo_delito_snic_id,cod_delito,codigo_delito_snic_nombre,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
48531,6,Buenos Aires,6260,Esteban Echeverría,2006,1,1,homicidios dolosos,19,20.0,17.0,3.0,0.0,6.912761,7.276590,NaN,NaN
92841,6,Buenos Aires,6553,Monte Hermoso,2005,2,2,homicidios dolosos en grado de tentativa,1,1.0,1.0,0.0,0.0,17.834848,17.834848,NaN,NaN
495574,94,"Tierra del Fuego, Antártida e Islas del Atlánt...",94014,Ushuaia,2012,4,4,homicidios culposos por otros hechos,11,11.0,3.0,8.0,0.0,17.587055,17.587055,9.415309,26.073071
201665,18,Corrientes,18154,San Miguel,2005,4,4,homicidios culposos por otros hechos,0,0.0,0.0,0.0,0.0,0.000000,0.000000,NaN,NaN
134173,6,Buenos Aires,6840,Tres de Febrero,2017,4,4,homicidios culposos por otros hechos,3,3.0,1.0,0.0,2.0,0.872303,0.872303,0.608361,0.000000


In [46]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38379 entries, 0 to 497031
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   provincia_id               38379 non-null  int64  
 1   provincia_nombre           38379 non-null  object 
 2   departamento_id            38379 non-null  int64  
 3   departamento_nombre        38379 non-null  object 
 4   anio                       38379 non-null  int64  
 5   codigo_delito_snic_id      38379 non-null  object 
 6   cod_delito                 38379 non-null  int64  
 7   codigo_delito_snic_nombre  38379 non-null  object 
 8   cantidad_hechos            38379 non-null  int64  
 9   cantidad_victimas          38379 non-null  float64
 10  cantidad_victimas_masc     38379 non-null  float64
 11  cantidad_victimas_fem      38379 non-null  float64
 12  cantidad_victimas_sd       38379 non-null  float64
 13  tasa_hechos                36648 non-null  float64

In [45]:
# tipos de delitos
df_homicidios['codigo_delito_snic_nombre'].unique()

array(['homicidios dolosos', 'homicidios dolosos en grado de tentativa',
       'homicidios culposos por otros hechos'], dtype=object)